In [5]:
import numpy as np
import pymc as pm

# datele initiale
publicity = np.array([1.5, 2.0, 2.3, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0,
                      6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0])
sales = np.array([5.2, 6.8, 7.5, 8.0, 9.0, 10.2, 11.5, 12.0, 13.5, 14.0,
                  15.0, 15.5, 16.2, 17.0, 18.0, 18.5, 19.5, 20.0, 21.0, 22.0])

# modelul Bayesian: y ~ Normal(α + β * x, σ)
with pm.Model() as model:
    alpha = pm.Normal("alpha", mu=0, sigma=10)   # intercept (variabila aleatoare)
    beta  = pm.Normal("beta",  mu=0, sigma=10)   # panta
    sigma = pm.HalfNormal("sigma", sigma=5)      # zgomot (deviatia standarda erorilor (noise): σ)
    mu = alpha + beta * publicity                # media liniara

    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=sales) #datele observate vin dintr-o distributie normala

    # sampling din posterior
    posterior = pm.sample(
        1000, tune=1000, chains=1, cores=1,
        random_seed=42, progressbar=False
    )

def get_samples(obj, name):
    try:
        return obj.posterior[name].values.reshape(-1)
    except AttributeError:
        return np.asarray(obj.get_values(name, combine=True))

alpha_s = get_samples(posterior, "alpha")
beta_s  = get_samples(posterior, "beta")
sigma_s = get_samples(posterior, "sigma")

#a) estimarea coeficientilor (media posteriorului, intercept si slope)
alpha_mean = alpha_s.mean()
beta_mean  = beta_s.mean()
sigma_mean = sigma_s.mean()

print("Posterior mean estimates:")
print(f"  intercept (alpha) ≈ {alpha_mean:.3f}")
print(f"  slope     (beta)  ≈ {beta_mean:.3f}")
print(f"  noise sd  (sigma) ≈ {sigma_mean:.3f}")

# b) intervalele credibile 95%
def ci95(samples):
    return np.percentile(samples, [2.5, 97.5])

alpha_ci = ci95(alpha_s)
beta_ci  = ci95(beta_s)
sigma_ci = ci95(sigma_s)

print("\n95% credible intervals:")
print(f"  alpha: [{alpha_ci[0]:.3f}, {alpha_ci[1]:.3f}]")
print(f"  beta : [{beta_ci[0]:.3f}, {beta_ci[1]:.3f}]")
print(f"  sigma: [{sigma_ci[0]:.3f}, {sigma_ci[1]:.3f}]")

# 5. c) predictii pentru noile niveluri de publicitate
new_publicity = np.array([2.0, 5.0, 8.0, 11.0])

print("\nPredicted future sales (in thousands of dollars):")
for x in new_publicity:
    # pentru fiecare sample din posterior construim o valoare posibila a vanzarilor
    mean_line = alpha_s + beta_s * x
    y_pred_s = np.random.normal(loc=mean_line, scale=sigma_s)
    m = y_pred_s.mean()
    low, high = ci95(y_pred_s)
    print(f"  publicity = {x:4.1f} -> "
          f"sales ≈ {m:.2f} "
          f"(95% predictive interval [{low:.2f}, {high:.2f}])")


Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, sigma]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 29 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks


Posterior mean estimates:
  intercept (alpha) ≈ 3.381
  slope     (beta)  ≈ 1.705
  noise sd  (sigma) ≈ 0.401

95% credible intervals:
  alpha: [2.936, 3.809]
  beta : [1.643, 1.767]
  sigma: [0.282, 0.578]

Predicted future sales (in thousands of dollars):
  publicity =  2.0 -> sales ≈ 6.80 (95% predictive interval [5.94, 7.66])
  publicity =  5.0 -> sales ≈ 11.90 (95% predictive interval [11.15, 12.69])
  publicity =  8.0 -> sales ≈ 17.01 (95% predictive interval [16.21, 17.87])
  publicity = 11.0 -> sales ≈ 22.14 (95% predictive interval [21.26, 23.05])
